In [60]:
#!pip install pulp
import pandas as pd


We load the user expected return and user sectoral HHI that other python notebook calculates and saves

In [616]:
# Read target_hhi the text file
with open('user_er.txt', 'r') as file:
    user_er = float(file.read())

print("User expected return : ",user_er)
    
# Read target_hhi the text file
with open('target_hhi.txt', 'r') as file:
    target_hhi = float(file.read())

print("\nTarget HHI : ",target_hhi)


User expected return :  0.16304618820512823

Target HHI :  343.32962842598647


We load the Index fund data in pivoted format. We prepare this format in other python notebook. This dataframe shows the stock proportion in each Index fund. 

In [617]:
index_df = pd.read_csv('./Data/Index_Data/index_fund_pivoted_weights.csv',index_col=False)
index_df.head()


,STOCK_ID,ID1,ID10,ID11,ID12,ID13,ID14,ID15,ID16,ID17,...,ID34,ID35,ID36,ID37,ID4,ID5,ID6,ID7,ID8,ID9
0,A1,0.000000,0.000000,0.0,0.0,0.009632,0.0,0.002124,0.0,0.0,...,0.0,0.001818,0.000000,0.0,0.001889,0.0,0.0,0.0,0.000000,0.0
1,A2,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000147,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
2,N1,0.004605,0.028019,0.0,0.0,0.000000,0.0,0.003573,0.0,0.0,...,0.0,0.003053,0.004981,0.0,0.003180,0.0,0.0,0.0,0.018733,0.0
3,N10,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000434,0.000000,0.0,0.000451,0.0,0.0,0.0,0.000000,0.0
4,N100,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000759,0.000000,0.0,0.000790,0.0,0.0,0.0,0.000000,0.0


Similarly we load Index fund wise sectoral break up; which is also prepared in other python notebook.

In [618]:
index_df_sec = pd.read_csv('./Data/Index_Data/index_fund_sector_pivoted_weights.csv')
index_df_sec.head()


,SECTOR_ID,ID1,ID10,ID11,ID12,ID13,ID14,ID15,ID16,ID17,...,ID34,ID35,ID36,ID37,ID4,ID5,ID6,ID7,ID8,ID9
0,SEC1,0.000000,0.0,0.0,0.0,0.0,0.0,0.001592,0.000000,0.0,...,0.000000,0.002083,0.000000,0.0,0.002164,0.00000,0.0,0.0,0.0,0.0
1,SEC100,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.198287,0.0,...,0.013317,0.000494,0.000000,0.0,0.000355,0.00000,0.0,0.0,0.0,0.0
2,SEC101,0.000000,0.0,0.0,0.0,0.0,0.0,0.001211,0.531109,0.0,...,0.000000,0.001323,0.000000,0.0,0.001288,0.00000,0.0,0.0,0.0,0.0
3,SEC102,0.004075,0.0,0.0,0.0,0.0,0.0,0.004690,0.000000,0.0,...,0.000000,0.004352,0.004434,0.0,0.004165,0.01022,0.0,0.0,0.0,0.0
4,SEC103,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000230,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


Finally, we load melted user portfolio. This is prepared in other python notebook by taking the fund names and then using holding data, 'melt' it at stock level.

In [619]:
#user_file_path = './Data/USER_MF_MELTED_PF.xlsx'
user_file_path = './Data/USER_MF_MELTED_PF_V2.csv'

#user_df = pd.read_excel(user_file_path, sheet_name='MELTED_PF')
user_df = pd.read_csv(user_file_path)
user_df = user_df[['STOCK_ID','USER_WEIGHT']]
print(user_df)


    STOCK_ID  USER_WEIGHT
0       S282     0.000923
1       S894     0.000584
2      S1241     0.000149
3      S1068     0.000038
4       S317     0.002435
..       ...          ...
590    S1230     0.000231
591     S649     0.000346
592     S287     0.005037
593     S646     0.001187
594     S648     0.000169

[595 rows x 2 columns]


In next few cells, we are trying to fill the gap between user stocks and universe of stocks coming from index funds. This gap will be filled by weights of 0. And at the end we would have same universe of stocks in user data and in index fund data. That makes it easier to compare.

In [620]:
#stocks which are in Index funds but not present in user portfolio should have weight of 0
#similarly stocks not in index funds but present in user portfolio, should be added to Index funds with 0 weights

unique_user_stocks  = set(user_df['STOCK_ID'].unique())
unique_index_stocks = set(index_df['STOCK_ID'].unique())
all_unique_stocks   = unique_user_stocks.union(unique_index_stocks)

In [621]:

# Find stocks in index funds but not in user portfolio and add them to df with weight 0
missing_in_user = all_unique_stocks - unique_user_stocks
missing_user_df = pd.DataFrame({'STOCK_ID': list(missing_in_user), 'USER_WEIGHT': [0] * len(missing_in_user)})

# Append missing stocks to the user portfolio data
user_df = pd.concat([user_df, missing_user_df])

print(user_df)


    STOCK_ID  USER_WEIGHT
0       S282     0.000923
1       S894     0.000584
2      S1241     0.000149
3      S1068     0.000038
4       S317     0.002435
..       ...          ...
326    S1111     0.000000
327     S613     0.000000
328     S976     0.000000
329    S1186     0.000000
330     S668     0.000000

[926 rows x 2 columns]


In [622]:
# Find stocks in user portfolio but not in index funds and add them to pivot_df with weight 0
missing_in_index = all_unique_stocks - unique_index_stocks

# Create a DataFrame for missing stocks with weights set to 0
missing_df = pd.DataFrame({'STOCK_ID': list(missing_in_index)})

# Add columns for each FUND_ID in pivot_df with weights set to 0
for col in index_df.columns:
    if col != 'STOCK_ID':
        missing_df[col] = 0

# Concatenate the original pivot_df with the missing_df
index_df = pd.concat([index_df, missing_df], ignore_index=True)

# Fill missing values with 0 (if any)
index_df = index_df.fillna(0)

# Display the resulting DataFrame
print(index_df)

# Optionally, save the pivot table to a CSV file
#index_df.to_csv('./Data/Index_Data/index_fund_pivoted_weights.csv', index=False)


    STOCK_ID       ID1      ID10  ID11  ID12      ID13  ID14      ID15  ID16  \
0         A1  0.000000  0.000000   0.0   0.0  0.009632   0.0  0.002124   0.0   
1         A2  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
2         N1  0.004605  0.028019   0.0   0.0  0.000000   0.0  0.003573   0.0   
3        N10  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
4       N100  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
..       ...       ...       ...   ...   ...       ...   ...       ...   ...   
921     S588  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
922     S447  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
923    S1043  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
924     S881  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   
925     S890  0.000000  0.000000   0.0   0.0  0.000000   0.0  0.000000   0.0   

     ID17  ...  ID34      ID35      ID3

Next we load index fund return data and transpose it. We will use this data in our constraint that we want to mske portfolio of Index funds which have higher expected return that user's own existing portfolio.

In [623]:
fund_er             =  pd.read_csv('./Data/Index_Data/mapping/index_fund_map.csv')
fund_er             =  fund_er[['FUND_ID','EXP_RET']]
fund_er             = fund_er.set_index('FUND_ID').transpose()
fund_er             = pd.DataFrame(fund_er)
fund_er.head()

FUND_ID,ID1,ID2,ID3,ID4,ID5,ID6,ID7,ID8,ID9,ID10,...,ID28,ID29,ID30,ID31,ID32,ID33,ID34,ID35,ID36,ID37
EXP_RET,0.176097,0.181787,0.158786,0.188228,0.333512,0.158869,0.123422,0.154788,0.131021,0.138195,...,0.171217,0.050993,0.136478,0.273519,0.217424,0.220379,0.218784,0.190322,0.163126,0.123068


Preparing Data for pulp optimizer.We have following idea:

1 Our constraint of increasing overlap is linear; so possible to include in pulp program
2 Our constraint of increasing expected return is also linear; that is also possible to include in pulp program
3 However, the sectoral HHI calculation involves squaring the sectoral weights. This is not possible in pulp program.

Hence we start our optimizer ignoring the sectoral HHI constraint and get a solution. Then Taking it as initial solution, we iterate through a hueristic algorithm to improve solution. Here in hueristic algorithm, we can include HHI improvement goal. We want to also show the possibility that using this method, we can include any non-linear and complex optimization goal such as VaR, volatility, liquidity and so on. 

In [624]:
import numpy as np

# Extract stock IDs and fund IDs
stock_ids = index_df['STOCK_ID'].values
fund_ids = index_df.columns[1:].values
sec_ids  = index_df_sec['SECTOR_ID'].values

# Create a dictionary mapping stock IDs to target weights
target_weights = user_df.set_index('STOCK_ID')['USER_WEIGHT'].to_dict()



In [625]:
from pulp import *

# Create a PuLP problem
prob = LpProblem("StockWeights", LpMinimize)

# Define the proportions as variables
proportions = LpVariable.dicts("p", fund_ids, lowBound=0)

# Define slack and surplus variables
slack = LpVariable.dicts("slack", stock_ids, lowBound=0)
surplus = LpVariable.dicts("surplus", stock_ids, lowBound=0)
slack_er = LpVariable("slack_er", lowBound=-0.1)

# Define the objective function to minimize the sum of slack and surplus variables
#prob += lpSum(slack[stock_id] + surplus[stock_id] for stock_id in stock_ids) + slack_er, "Minimize slack and surplus"

prob += lpSum(slack[stock_id] + surplus[stock_id] for stock_id in stock_ids), "Minimize slack and surplus"


# Define the constraints for each stock
for stock_id in stock_ids:
    prob += (lpSum([proportions[fund_id] * index_df.loc[index_df['STOCK_ID'] == stock_id, fund_id].values[0] for fund_id in fund_ids])
             - slack[stock_id] + surplus[stock_id] == target_weights[stock_id] )

#weighted average of expected return should be greater than or equal to user expected return 
prob += (lpSum([proportions[fund_id] * fund_er[fund_id].values[0] for fund_id in fund_ids])
             - slack_er  == user_er )

    
# Constraint: total weights of all funds should sum up to 1
prob += lpSum(proportions[fund_id] for fund_id in fund_ids) == 1, "Total weight constraint"


#print(prob)

# Solve the problem
prob.solve()

# Print the results and update the proportions dictionary
proportions_dict = {}
for v in prob.variables():
    print(v.name, "=", v.varValue)
    if v.name.startswith("p_"):
        fund_id = v.name.split("_")[1]
        proportions_dict[fund_id] = v.varValue


p_ID1 = 0.0
p_ID10 = 0.0
p_ID11 = 0.0021959401
p_ID12 = 0.009872905
p_ID13 = 0.0
p_ID14 = 0.0
p_ID15 = 0.32411751
p_ID16 = 0.0
p_ID17 = 0.0
p_ID18 = 0.015830207
p_ID19 = 0.0
p_ID2 = 0.0
p_ID20 = 0.026864043
p_ID21 = 0.0
p_ID22 = 0.0
p_ID23 = 0.0
p_ID24 = 0.19846958
p_ID25 = 0.0
p_ID26 = 0.039878008
p_ID27 = 0.019877805
p_ID28 = 0.0
p_ID29 = 0.0
p_ID3 = 0.0
p_ID30 = 0.089508632
p_ID31 = 0.0
p_ID32 = 0.0
p_ID33 = 0.0
p_ID34 = 0.033648886
p_ID35 = 0.0
p_ID36 = 0.053567041
p_ID37 = 0.0
p_ID4 = 0.13111621
p_ID5 = 0.0
p_ID6 = 0.048354967
p_ID7 = 0.0
p_ID8 = 0.0
p_ID9 = 0.0066982685
slack_A1 = 0.0030831476
slack_A2 = 6.0750454e-05
slack_N1 = 0.0024764233
slack_N10 = 0.00034857895
slack_N100 = 0.00060995947
slack_N101 = 5.5104735e-05
slack_N102 = 0.00033895744
slack_N103 = 6.1578096e-05
slack_N104 = 4.8051736e-05
slack_N105 = 5.3857436e-05
slack_N106 = 0.00014751118
slack_N107 = 0.00031175303
slack_N109 = -0.0
slack_N110 = 0.00019387439
slack_N111 = 0.00012031722
slack_N113 = 0.0034245528
slac

In [626]:
prob

StockWeights:
MINIMIZE
1*slack_A1 + 1*slack_A2 + 1*slack_N1 + 1*slack_N10 + 1*slack_N100 + 1*slack_N101 + 1*slack_N102 + 1*slack_N103 + 1*slack_N104 + 1*slack_N105 + 1*slack_N106 + 1*slack_N107 + 1*slack_N109 + 1*slack_N110 + 1*slack_N111 + 1*slack_N113 + 1*slack_N114 + 1*slack_N115 + 1*slack_N116 + 1*slack_N117 + 1*slack_N118 + 1*slack_N121 + 1*slack_N127 + 1*slack_N130 + 1*slack_N131 + 1*slack_N132 + 1*slack_N133 + 1*slack_N137 + 1*slack_N138 + 1*slack_N142 + 1*slack_N143 + 1*slack_N144 + 1*slack_N147 + 1*slack_N148 + 1*slack_N150 + 1*slack_N151 + 1*slack_N152 + 1*slack_N153 + 1*slack_N156 + 1*slack_N157 + 1*slack_N158 + 1*slack_N159 + 1*slack_N160 + 1*slack_N161 + 1*slack_N162 + 1*slack_N163 + 1*slack_N164 + 1*slack_N165 + 1*slack_N168 + 1*slack_N17 + 1*slack_N171 + 1*slack_N173 + 1*slack_N175 + 1*slack_N177 + 1*slack_N178 + 1*slack_N18 + 1*slack_N180 + 1*slack_N182 + 1*slack_N183 + 1*slack_N184 + 1*slack_N185 + 1*slack_N187 + 1*slack_N188 + 1*slack_N19 + 1*slack_N192 + 1*slack_N193

Printing the output where initial solution recommends the weights.

In [627]:
# Filter the dictionary to get only key-value pairs with non-zero values
inv_proportion = {k: v for k, v in proportions_dict.items() if v != 0}
print(inv_proportion)


{'ID11': 0.0021959401, 'ID12': 0.009872905, 'ID15': 0.32411751, 'ID18': 0.015830207, 'ID20': 0.026864043, 'ID24': 0.19846958, 'ID26': 0.039878008, 'ID27': 0.019877805, 'ID30': 0.089508632, 'ID34': 0.033648886, 'ID36': 0.053567041, 'ID4': 0.13111621, 'ID6': 0.048354967, 'ID9': 0.0066982685}


We create a function to calculate overlap between the portfolio we created using index funds and user portfolio

In [628]:
# Function to calculate sum of minimum stock weights in portfolio and target weight
def calc_overlap(df_stock_weights, df_target_weights, proportions):
    
    total_sum = 0
    
    # Iterate through each stock in the stock weights dataframe
    for index, row in df_stock_weights.iterrows():
        stock_id = row['STOCK_ID']
        
        # Calculate the stock weight in the portfolio
        stock_weight_in_portfolio = sum(row[fund_id] * proportions[fund_id] for fund_id in proportions.keys())
        
        # Get the target weight of the stock
        target_weight = target_weights.get(stock_id, 0)
        
        # Add the minimum of the stock weight in portfolio and target weight to the total sum
        total_sum += min(stock_weight_in_portfolio, target_weight)
        
    
    return total_sum

# Calculate the sum of minimum stock weights
overlap = calc_overlap(index_df, user_df, proportions_dict)
print("Total overlap managed:", overlap)


Total overlap managed: 0.6198286544530938


Create a function which calculated the expected return based on the propertion we give and expected return data of each funds that we give

In [629]:
def calc_expected_return(fund_er, proportions):
    er = 0
    for fund_id in fund_ids:
        er += (fund_er[fund_id].values[0] * proportions[fund_id])
    return(er)


exp_er  = calc_expected_return(fund_er, proportions_dict)
print("Expected return managed ", exp_er)

Expected return managed  0.18629749184767802


Following function calculated the sectoral HHI

In [630]:
def get_sector_hhi(proportions_dict,index_df_sec):
    proportions_df      = pd.DataFrame.from_dict(proportions_dict, orient='index', columns=['FUND_WEIGHT'])
    proportions_df      = proportions_df.reset_index().rename(columns={'index': 'FUND_ID'})

    index_df_sec        = pd.merge(index_df_sec,proportions_df,on= 'FUND_ID', how= 'left')
    index_df_sec.head()

    index_df_sec['TOTAL_WEIGHT'] = index_df_sec['WEIGHT']*index_df_sec['FUND_WEIGHT']
    sec_weight                   = index_df_sec.groupby('SECTOR_ID')['TOTAL_WEIGHT'].sum()
    sec_weight                   = sec_weight.reset_index()
    sec_weight['SQ_WEIGHT']      = (sec_weight['TOTAL_WEIGHT']*100)*(sec_weight['TOTAL_WEIGHT']*100)
    return(sec_weight['SQ_WEIGHT'].sum())

index_df_sec            = pd.read_csv('./Data/Index_Data/index_fund_sector_weights.csv')
sec_hhi                 = get_sector_hhi(proportions_dict,index_df_sec)
sec_hhi

392.5659529457542

Now we prepare for hueristics. Plan is simple. Take the solution that we got from the pulp program as initial solution and randomly change the proportion to get a new solution and if it is improvement, keep it else iterate over next. We also have put checks to avoid same keys that we changed just now. 

In [631]:
import random
import matplotlib.pyplot as plt


def get_a_new_sol(current_sol,exclude_keys=None):
    
    if exclude_keys is None:
        exclude_keys = []
  
    step_size     = round(random.uniform(0.01, 0.05), 4)
    
    # Create a list of keys where the value is more than 0.01
    reduce_keys = [k for k, v in current_sol.items() if v > step_size and k not in exclude_keys and v > 0]
    
    # Create a list of keys where the value is less than 0.99
    increase_keys = [k for k, v in current_sol.items() if v < (1-step_size) and k not in exclude_keys and v > 0]
    
    # Ensure there are valid keys to adjust
    if reduce_keys and increase_keys:
        
        # Randomly choose a key to reduce and a key to increase
        reduce_key   = random.choice(reduce_keys)
        increase_key = random.choice(increase_keys)
        
        # Ensure the chosen keys are different
        if reduce_key != increase_key:
            # Reduce 0.01 from the selected reduce_key and add 0.01 to the selected increase_key
            current_sol[reduce_key] = current_sol[reduce_key] - step_size
            current_sol[increase_key] = current_sol[increase_key] + step_size
    
            # Print the adjustments
            #print(f"Reduced 0.01 from {reduce_key}, new value: {current_sol[reduce_key]}")
            #print(f"Increased 0.01 to {increase_key}, new value: {current_sol[increase_key]}")
        
    return current_sol,(reduce_key, increase_key)



In [633]:
nr_iterations   = 100

# Initial run
exclude_keys = []

saved_proportions         = {key: value for key, value in proportions_dict.items()}
current_sol = {key: value for key, value in proportions_dict.items()}

#print("saved proportions is \n\n", saved_proportions)


for i in range(nr_iterations):

    current_sol, exclude_keys = get_a_new_sol(current_sol, exclude_keys)
    
    new_overlap      = calc_overlap(index_df, user_df, current_sol)      
    new_er           = calc_expected_return(fund_er, current_sol)
    new_hhi          = get_sector_hhi(current_sol,index_df_sec)

    improvement      = 100*(new_overlap- overlap) + 150*(new_er-exp_er) - (new_hhi-sec_hhi)/100 
    print("Imporvement scope : ",improvement," iteration : ", i)
    if improvement > 0:
        #proportions_dict = copy.deepcopy(current_sol)
        saved_proportions = {key: value for key, value in current_sol.items()}
        print("Proportions improved a bit")
        print("New overlap ", new_overlap)
        print("New expected return ", new_er)
        print("New hhi ", new_hhi)
    else:
        current_sol = {key: value for key, value in proportions_dict.items()}
        
        

Imporvement scope :  -0.19490165450949515  iteration :  0
Imporvement scope :  -2.5300570981670467  iteration :  1
Imporvement scope :  -3.2571942696944385  iteration :  2
Imporvement scope :  -3.7245396087746725  iteration :  3
Imporvement scope :  -3.9888792152208077  iteration :  4
Imporvement scope :  -3.716898623015038  iteration :  5
Imporvement scope :  -2.948376035108022  iteration :  6
Imporvement scope :  -3.289551026772714  iteration :  7
Imporvement scope :  -4.022968716855596  iteration :  8
Imporvement scope :  -4.022968716855596  iteration :  9


Print final proportions and results.

In [634]:
# Convert proportions to percentages, filter out zero values, and add % sign
inv_proportion = {k: f"{round(v * 100, 2)}%" for k, v in saved_proportions.items() if v != 0}

# Print the final proportions
print("Guys here is what we propose \n\n", inv_proportion)

new_overlap      = calc_overlap(index_df, user_df, saved_proportions)      
new_er           = calc_expected_return(fund_er, saved_proportions)
new_hhi          = get_sector_hhi(saved_proportions,index_df_sec)



print("\n\nWe bring in alpha of :", round((new_er - user_er)*100,2), "%")
print("Overlap with your existing portfolio would still be", round(new_overlap*100,2), "%")
print("And your sectoral HHI had to move from", round(target_hhi), "to",round(new_hhi))


Guys here is what we propose 

 {'ID11': '0.22%', 'ID12': '0.99%', 'ID15': '32.41%', 'ID18': '3.36%', 'ID20': '2.69%', 'ID24': '19.85%', 'ID26': '3.99%', 'ID27': '1.99%', 'ID30': '8.95%', 'ID34': '1.58%', 'ID36': '5.36%', 'ID4': '13.11%', 'ID6': '4.84%', 'ID9': '0.67%'}


We bring in alpha of : 2.32 %
Overlap with your existing portfolio would still be 61.8 %
And your sectoral HHI had to move from 343 to 390
